# Postprocessing data from the reanalysis of the COVID-19 multiomics project

As part of the case study associated with the data found in https://dx.doi.org/10.1101%2F2020.07.17.20156513, we compare the features discovered by the authors and us.

## Summary of our analysis

With our pipeline, we analysed the omics data from the original publication, comparing the severity of COVID-19 states (less severe vs more severe) according to the previously described `HFD-45` metric by the authors. We obtained informative features both within and across the omics data. Then, we compare the features identified from our single-omics analyses, multi-omics analyses, and the authors results.

In [1]:
import os
import re
import pandas as pd

In [2]:
# setup all file paths
data_dir = "../../data/MSV000085703/"
results_dir = "../../results/MSV000085703/MSV_4M_01/"

original_path = "/".join(["../../results/MSV000085703", "supp_table_3.txt"])

l_fmap_path = "/".join([data_dir, "lipidomics_featuremap.tsv"]) 
md_fmap_path = "/".join([data_dir, "metabolomicsdiscovery_featuremap.tsv"]) 
mt_fmap_path = "/".join([data_dir, "metabolomicstargeted_featuremap.tsv"]) 
p_fmap_path = "/".join([data_dir, "proteomics_featuremap.tsv"]) 
t_fmap_path = "/".join([data_dir, "transcriptomics_featuremap.tsv"]) 

l_splsda_path = "/".join([results_dir, "lipidomics_sPLSDA_max.txt"])
m_splsda_path = "/".join([results_dir, "metabolomics_sPLSDA_max.txt"])
p_splsda_path = "/".join([results_dir, "proteomics_sPLSDA_max.txt"])
t_splsda_path = "/".join([results_dir, "transcriptomics_sPLSDA_max.txt"])

l_diablo_path = "/".join([results_dir, "lipidomics_DIABLO_max.txt"])
m_diablo_path = "/".join([results_dir, "metabolomics_DIABLO_max.txt"])
p_diablo_path = "/".join([results_dir, "proteomics_DIABLO_max.txt"])
t_diablo_path = "/".join([results_dir, "transcriptomics_DIABLO_max.txt"])

diablo_path = "/".join([results_dir, "DIABLO_var_keepx_correlations.txt"])

In [3]:
# load the single omics data
l_splsda = pd.read_csv(l_splsda_path, sep="\t")
m_splsda = pd.read_csv(m_splsda_path, sep="\t")
p_splsda = pd.read_csv(p_splsda_path, sep="\t")
t_splsda = pd.read_csv(t_splsda_path, sep="\t")

# load the multi omics data
l_diablo = pd.read_csv(l_diablo_path, sep="\t")
m_diablo = pd.read_csv(m_diablo_path, sep="\t")
p_diablo = pd.read_csv(p_diablo_path, sep="\t")
t_diablo = pd.read_csv(t_diablo_path, sep="\t")

# load the diablo correlations
diablo = pd.read_csv(diablo_path, sep="\t")

In [4]:
# for technical reasons, we shortened names
# we saved the mappings before running the pipeline, now we remap them
l_fmap = pd.read_csv(l_fmap_path, sep="\t")[["short", "long"]].set_index("short")
md_fmap = pd.read_csv(md_fmap_path, sep="\t")[["short", "long"]].set_index("short")
mt_fmap = pd.read_csv(mt_fmap_path, sep="\t")[["short", "long"]].set_index("short")
p_fmap = pd.read_csv(p_fmap_path, sep="\t")[["short", "long"]].set_index("short")
t_fmap = pd.read_csv(t_fmap_path, sep="\t")[["short", "long"]].set_index("short")
fmap = pd.concat([l_fmap, md_fmap, mt_fmap, p_fmap, t_fmap])
fmap.index.name = None

# R quietly converts non-alphanumeric characters in feature names to "."
# we need to match these changes in order to remap
fmap.index = [re.sub('[^0-9a-zA-Z_]', '.', x) for x in fmap.index]

In [5]:
def remap_identifiers(data, keys):
    data = keys.merge(data, left_index=True, right_index=True, how="right")
    data.set_index("long", inplace=True)
    data.sort_values("importance", inplace=True, ascending=False)
    data.index.name = None
    return data[["GroupContrib", "importance"]]

singleomics = [remap_identifiers(x, fmap) for x in [l_splsda, m_splsda, p_splsda, t_splsda]]
# multiomics = [remap_identifiers(x, fmap) for x in [l_diablo, m_diablo, p_diablo, t_diablo]]

In [6]:
def compare_data(old, new):
    return old.merge(new, left_index=True, right_index=True, how="left")

# load in the most informative features identified by the original analysis
original = pd.read_csv(original_path, sep="\t").set_index("standardized_name")
original.index.name = None
original.index = [x.strip() for x in original.index.tolist()]

# investigate how many single omics features match the authors data
[compare_data(original, x).importance.nunique() for x in singleomics]

[3, 3, 6, 0]

In [7]:
# reformat the correlation matrix to a more readable format
diablo_flat = diablo.stack().reset_index()
diablo_flat = diablo_flat.merge(fmap, left_on="level_0", right_index=True, how="left")
diablo_flat.drop(["level_0"], axis=1, inplace=True)
diablo_flat = diablo_flat.merge(fmap, left_on="level_1", right_index=True, how="left")
diablo_flat.drop(["level_1"], axis=1, inplace=True)
diablo_flat = diablo_flat[["long_x", "long_y", 0]].sort_values(0, ascending=False)
diablo_flat.columns = ["Feature_1", "Feature_2", "Correlation"]
diablo_flat

,Feature_1,Feature_2,Correlation
1216,unknown RT 16.698449,unknown RT 16.698449,0.936421
4591,PGD,unknown RT 16.698449,0.930865
1261,unknown RT 16.698449,PGD,0.930865
4636,PGD,PGD,0.928084
2812,ANO10,ANO10,0.924443
...,...,...,...
3766,HHAT,unknown RT 16.698449,-0.897459
5611,ZNF527,PGD,-0.897917
4649,PGD,ZNF527,-0.897917
1274,unknown RT 16.698449,ZNF527,-0.906509


In [8]:
# do our highly correlating values match the authors?
feature_list = diablo_flat["Feature_1"].unique()#.str.strip().tolist()

feat_1 = original.merge(diablo_flat, left_index=True, right_on="Feature_1", how="left")
feat_1 = feat_1[feat_1.Correlation.notnull()]

feat_2 = original.merge(diablo_flat, left_index=True, right_on="Feature_2", how="left")
feat_2 = feat_2[feat_2.Correlation.notnull()]

feat_intersect = pd.concat([feat_1, feat_2]).reset_index().drop("index", axis=1)
feat_intersect.biomolecule_id.nunique()

1

# Notes

The files in the results directory were a concatenation of all features obtained from the analyses. These are available in gitlab in the results directory. To reproduce these files, after running our pipeline navigate to your results directory and run the following commands.

```
head -n1 lipidomics_1_sPLSDA_max.txt > lipidomics_sPLSDA_max.txt
for i in lipidomics_[0-9]_sPLSDA_max*; do tail -n +2 $i >> lipidomics_sPLSDA_max.txt; done
head -n1 metabolomics_1_sPLSDA_max.txt > metabolomics_sPLSDA_max.txt
for i in metabolomics_[0-9]_sPLSDA_max*; do tail -n +2 $i >> metabolomics_sPLSDA_max.txt; done
head -n1 proteomics_1_sPLSDA_max.txt > proteomics_sPLSDA_max.txt
for i in proteomics_[0-9]_sPLSDA_max*; do tail -n +2 $i >> proteomics_sPLSDA_max.txt; done
head -n1 transcriptomics_1_sPLSDA_max.txt > transcriptomics_sPLSDA_max.txt
for i in transcriptomics_[0-9]_sPLSDA_max*; do tail -n +2 $i >> transcriptomics_sPLSDA_max.txt; done

head -n1 lipidomics_1_DIABLO_var_keepx_max.txt > lipidomics_DIABLO_max.txt
for i in lipidomics_[0-9]_DIABLO_var_keepx_max*txt; do tail -n +2 $i >> lipidomics_DIABLO_max.txt; done
head -n1 metabolomics_1_DIABLO_var_keepx_max.txt > metabolomics_DIABLO_max.txt
for i in metabolomics_[0-9]_DIABLO_var_keepx_max*txt; do tail -n +2 $i >> metabolomics_DIABLO_max.txt; done
head -n1 proteomics_1_DIABLO_var_keepx_max.txt > proteomics_DIABLO_max.txt
for i in proteomics_[0-9]_DIABLO_var_keepx_max*txt; do tail -n +2 $i >> proteomics_DIABLO_max.txt; done
head -n1 transcriptomics_1_DIABLO_var_keepx_max.txt > transcriptomics_DIABLO_max.txt
for i in transcriptomics_[0-9]_DIABLO_var_keepx_max*txt; do tail -n +2 $i >> transcriptomics_DIABLO_max.txt; done
```